In [1]:
def get_text(text_file):
    index = text_file.index('_')
    text_new = text_file[index + 1:-4]
    
    return text_new

In [2]:
def get_list_file(index_dir):
    from os import listdir
    list_file_dir = sorted(listdir(index_dir))
    
    return list_file_dir

In [3]:
class wav_file:    
    
    def __init__(self, name_file_1, sample_rate_1, samples_1):
        self.name_file = name_file_1
        self.sample_rate = sample_rate_1
        self.samples = samples_1
        
    def __str__(self):
        return self.name_file

In [4]:
def get_file(index_file):    
    from scipy.io import wavfile
        
    sample_rate_1, samples_1 = wavfile.read(index_file)
    name_file_1 = get_text(index_file)
        
    return name_file_1, sample_rate_1, samples_1

In [5]:
import numpy as np
import math

from scipy.fftpack import fft
from scipy import signal

from IPython.display import Audio

from matplotlib import pyplot as plt

import random as rnd

Получение данных

In [6]:
m_wav_file = []
index_dir = "C:\\Users\\defaultuser0\\Word"

list_file = get_list_file(index_dir)

for file in list_file:
    index_file = index_dir + "\\" + file
    name_file_1, sample_rate_1, samples_1 = get_file(index_file)
    m_wav_file.append(wav_file(name_file_1, sample_rate_1, samples_1))

In [7]:
def m_wav_file_recreate(m_wav_file):
    m_wav_file_1 = []
    
    for file in m_wav_file:
        samples = file.samples
        name = file.name_file
        noise = np.random.normal(0,100,len(samples))
        samples = samples + noise
    
        sampl = []
        i = 0
        k = rnd.randint(20, 40)
        
        for pr in samples:
            sampl.append(pr)
            i += 1
            if i % k == 0:
                sampl.append(pr)
            
        m_wav_file_1.append(wav_file(name, 44100, sampl))
        
    return m_wav_file_1

In [8]:
m_wav_file_noise_sum = []

for i in range(10):
    print("Итерация", i + 1, "началась")
    m_wav_file_noise_sum += m_wav_file_recreate(m_wav_file)
    print("Итерация", i + 1, "закончилась")

print("Финальная итерация началась")
mY_wav_file_noise = m_wav_file_recreate(m_wav_file)
print("Финальная итерация завершена")

Итерация 1 началась
Итерация 1 закончилась
Итерация 2 началась
Итерация 2 закончилась
Итерация 3 началась
Итерация 3 закончилась
Итерация 4 началась
Итерация 4 закончилась
Итерация 5 началась
Итерация 5 закончилась
Итерация 6 началась
Итерация 6 закончилась
Итерация 7 началась
Итерация 7 закончилась
Итерация 8 началась
Итерация 8 закончилась
Итерация 9 началась
Итерация 9 закончилась
Итерация 10 началась
Итерация 10 закончилась
Финальная итерация началась
Финальная итерация завершена


In [9]:
sample_rate = 44100

In [9]:
def get_top_fft_segment(f):
    m = []
    
    f = f[:len(f)//2]

    for i in range(len(f)-11):
        if abs(f[i+6]) == max(f[i:i+11], key=abs) and abs(max(f[i:i+11], key=abs)) > 5000:
            m.append((i+6, f[i+6]))
    
    fft_top = sorted(sorted(m, key=lambda x: x[1])[-50:], key=lambda x: x[0])
    
    while len(fft_top) < 50:
        fft_top.append((0, 0))
    
    fft_result = []
    for ft in fft_top:
        fft_result += ft
    
    return fft_result

In [10]:
def fft_exp(samples):
    from scipy.fftpack import fft
    
    result_samples = []
    
    result_samples += [len(samples)]
    
    fft_complex_file = fft(samples).tolist()
    
    fft_abs_file = [abs(x) for x in fft_complex_file]
    result_samples += [min(fft_abs_file, key=abs)]
    result_samples += [sum(fft_abs_file)/len(fft_abs_file)]
    
    fft_real_file = [x.real for x in fft_complex_file]
    result_samples += [min(fft_real_file, key=abs)]
    result_samples += [sum(fft_real_file)/len(fft_real_file)]
    
    fft_imag_file = [x.imag for x in fft_complex_file]
    result_samples += [min(fft_imag_file, key=abs)]
    result_samples += [sum(fft_imag_file)/len(fft_imag_file)]
    
    fft_top_abs = get_top_fft_segment(fft_abs_file)
    fft_top_real = get_top_fft_segment(fft_real_file)
    fft_top_imag = get_top_fft_segment(fft_imag_file)
    
    result_samples += fft_top_abs
    result_samples += fft_top_real
    result_samples += fft_top_imag
    
    return result_samples

Преобразование обучающей выборки

In [11]:
X_train = []
Y_train = []

i = 0

for wav in m_wav_file_noise_sum:
    i += 1
    if i % 400 == 0:
        print(i)
        
    samples_1 = fft_exp(wav.samples)
    X_train.append(samples_1)
    Y_train.append(wav.name_file)

400
800
1200
1600
2000
2400
2800
3200
3600
4000
4400
4800
5200
5600
6000
6400
6800
7200
7600


Преобразование тестовых выборок

In [12]:
X_test = []
Y_test = []

i = 0
for wav in m_wav_file:
    i += 1
    if i % 100 == 0:
        print(i)
        
    samples_2 = fft_exp(wav.samples)
    
    Y_test.append(wav.name_file)
    X_test.append(samples_2)
    
X_noise_test = []
Y_noise_test = []

i = 0
for wav in mY_wav_file_noise:
    i += 1
    if i % 100 == 0:
        print(i)
        
    samples_3 = fft_exp(wav.samples)
    
    Y_noise_test.append(wav.name_file)
    X_noise_test.append(samples_3)

100
200
300
400
500
600
700
100
200
300
400
500
600
700


Нормализация

In [104]:
from sklearn.preprocessing import MinMaxScaler 

min_max_scaler = MinMaxScaler() 

X_train = min_max_scaler.fit_transform(X_train) 
X_test = min_max_scaler.transform(X_test)
X_noise_test = min_max_scaler.transform(X_noise_test)

Обучение

In [ ]:
from time import time
from sklearn import metrics

Логистическая регрессия

In [105]:
from sklearn.linear_model import LogisticRegression

t_start = time()
model = LogisticRegression()
model.fit(X_train, Y_train)
print(model)
t_finish = time()
print(t_finish - t_start)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
220.84639048576355


Результат по тестовой выборке

In [107]:
expected_1 = Y_test
predicted_1 = model.predict(X_test)

print(metrics.classification_report(expected_1, predicted_1))

                  precision    recall  f1-score   support

          Одесса       0.00      0.00      0.00         1
               а       0.00      0.00      0.00        13
         автобус       0.00      0.00      0.00         1
       автозавод       0.00      0.00      0.00         1
       автолавка       0.00      0.00      0.00         1
         агрегат       0.00      0.00      0.00         1
          актёра       0.00      0.00      0.00         1
         актёров       0.00      0.00      0.00         1
         артисты       0.00      0.00      0.00         1
            базу       0.00      0.00      0.00         1
             без       0.00      0.00      0.00         2
          берегу       0.00      0.00      0.00         1
             бит       0.00      0.00      0.00         2
            блюд       0.00      0.00      0.00         1
        больницу       0.00      0.00      0.00         1
         больных       0.00      0.00      0.00         1
            б

Y:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Результат по зашумленной тестовой выборке

In [108]:
expected_2 = Y_noise_test
predicted_2 = model.predict(X_noise_test)

print(metrics.classification_report(expected_2, predicted_2))

                  precision    recall  f1-score   support

          Одесса       1.00      1.00      1.00         1
               а       0.40      0.62      0.48        13
         автобус       1.00      1.00      1.00         1
       автозавод       1.00      1.00      1.00         1
       автолавка       1.00      1.00      1.00         1
         агрегат       1.00      1.00      1.00         1
          актёра       0.00      0.00      0.00         1
         актёров       0.00      0.00      0.00         1
         артисты       1.00      1.00      1.00         1
            базу       0.00      0.00      0.00         1
             без       0.00      0.00      0.00         2
          берегу       1.00      1.00      1.00         1
             бит       0.00      0.00      0.00         2
            блюд       0.00      0.00      0.00         1
        больницу       0.00      0.00      0.00         1
         больных       0.00      0.00      0.00         1
            б

Y:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Результат по исходным данным

In [109]:
expected_3 = Y_train
predicted_3 = model.predict(X_train)

print(metrics.classification_report(expected_3, predicted_3))

                  precision    recall  f1-score   support

          Одесса       1.00      1.00      1.00        10
               а       0.37      0.47      0.42       130
         автобус       1.00      1.00      1.00        10
       автозавод       1.00      1.00      1.00        10
       автолавка       1.00      1.00      1.00        10
         агрегат       1.00      1.00      1.00        10
          актёра       0.00      0.00      0.00        10
         актёров       1.00      0.30      0.46        10
         артисты       1.00      1.00      1.00        10
            базу       0.00      0.00      0.00        10
             без       0.00      0.00      0.00        20
          берегу       1.00      1.00      1.00        10
             бит       0.00      0.00      0.00        20
            блюд       0.00      0.00      0.00        10
        больницу       0.00      0.00      0.00        10
         больных       0.00      0.00      0.00        10
            б

Y:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Наивный Байесовский метод

In [110]:
from sklearn.naive_bayes import GaussianNB
t_start = time()
model_gb = GaussianNB()
model_gb.fit(X_train, Y_train)
t_finish = time()
print(t_finish - t_start)

0.6763448715209961


In [111]:
expected_gb = Y_test
predicted_gb = model_gb.predict(X_test)

print(metrics.classification_report(expected_gb, predicted_gb))

                  precision    recall  f1-score   support

          Одесса       0.00      0.00      0.00         1
               а       0.00      0.00      0.00        13
         автобус       0.00      0.00      0.00         1
       автозавод       0.00      0.00      0.00         1
       автолавка       0.00      0.00      0.00         1
         агрегат       0.00      0.00      0.00         1
          актёра       0.00      0.00      0.00         1
         актёров       0.00      0.00      0.00         1
         артисты       0.00      0.00      0.00         1
            базу       0.00      0.00      0.00         1
             без       0.00      0.00      0.00         2
          берегу       0.00      0.00      0.00         1
             бит       0.00      0.00      0.00         2
            блюд       0.00      0.00      0.00         1
        больницу       0.00      0.00      0.00         1
         больных       0.00      0.00      0.00         1
            б

Y:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [27]:
expected_gb_1 = Y_noise_test
predicted_gb_1 = model_gb.predict(X_noise_test)

print(metrics.classification_report(expected_gb_1, predicted_gb_1))

                  precision    recall  f1-score   support

          Одесса       1.00      1.00      1.00         1
               а       0.21      0.31      0.25        13
         автобус       1.00      1.00      1.00         1
       автозавод       1.00      1.00      1.00         1
       автолавка       1.00      1.00      1.00         1
         агрегат       1.00      1.00      1.00         1
          актёра       1.00      1.00      1.00         1
         актёров       1.00      1.00      1.00         1
         артисты       1.00      1.00      1.00         1
            базу       1.00      1.00      1.00         1
             без       1.00      0.50      0.67         2
          берегу       1.00      1.00      1.00         1
             бит       1.00      1.00      1.00         2
            блюд       1.00      1.00      1.00         1
        больницу       1.00      1.00      1.00         1
         больных       0.00      0.00      0.00         1
            б

Y:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [29]:
expected_gb_2 = Y_train
predicted_gb_2 = model_gb.predict(X_train)

print(metrics.classification_report(expected_gb_2, predicted_gb_2))

                  precision    recall  f1-score   support

          Одесса       1.00      1.00      1.00        10
               а       0.37      0.52      0.43       130
         автобус       1.00      1.00      1.00        10
       автозавод       1.00      1.00      1.00        10
       автолавка       1.00      1.00      1.00        10
         агрегат       1.00      1.00      1.00        10
          актёра       1.00      1.00      1.00        10
         актёров       1.00      1.00      1.00        10
         артисты       1.00      1.00      1.00        10
            базу       1.00      1.00      1.00        10
             без       1.00      0.50      0.67        20
          берегу       1.00      1.00      1.00        10
             бит       1.00      1.00      1.00        20
            блюд       1.00      1.00      1.00        10
        больницу       1.00      1.00      1.00        10
         больных       1.00      1.00      1.00        10
            б

Y:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Метод ближайших соседей

In [39]:
from sklearn.neighbors import KNeighborsClassifier

t_start = time()
model_kn = KNeighborsClassifier(11)
model_kn.fit(X_train, Y_train)
t_finish = time()
print(model_kn)
print(t_finish - t_start)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=11, p=2,
           weights='uniform')
0.43369174003601074


In [40]:
expected_kn = Y_test
predicted_kn = model_kn.predict(X_test)

print(metrics.classification_report(expected_kn, predicted_kn))

                  precision    recall  f1-score   support

          Одесса       1.00      1.00      1.00         1
               а       0.86      0.92      0.89        13
         автобус       1.00      1.00      1.00         1
       автозавод       1.00      1.00      1.00         1
       автолавка       1.00      1.00      1.00         1
         агрегат       1.00      1.00      1.00         1
          актёра       1.00      1.00      1.00         1
         актёров       1.00      1.00      1.00         1
         артисты       1.00      1.00      1.00         1
            базу       0.00      0.00      0.00         1
             без       1.00      1.00      1.00         2
          берегу       1.00      1.00      1.00         1
             бит       1.00      0.50      0.67         2
            блюд       1.00      1.00      1.00         1
        больницу       1.00      1.00      1.00         1
         больных       1.00      1.00      1.00         1
            б

Y:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [41]:
expected_kn_1 = Y_train
predicted_kn_1 = model_kn.predict(X_train)

print(metrics.classification_report(expected_kn_1, predicted_kn_1))

                  precision    recall  f1-score   support

          Одесса       0.91      1.00      0.95        10
               а       0.86      0.91      0.88       130
         автобус       0.50      0.70      0.58        10
       автозавод       0.75      0.90      0.82        10
       автолавка       0.77      1.00      0.87        10
         агрегат       0.89      0.80      0.84        10
          актёра       1.00      1.00      1.00        10
         актёров       1.00      1.00      1.00        10
         артисты       0.75      0.90      0.82        10
            базу       1.00      0.50      0.67        10
             без       0.38      0.90      0.53        20
          берегу       0.91      1.00      0.95        10
             бит       0.80      0.60      0.69        20
            блюд       0.58      0.70      0.64        10
        больницу       1.00      1.00      1.00        10
         больных       1.00      1.00      1.00        10
            б

Y:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [42]:
expected_kn_2 = Y_noise_test
predicted_kn_2 = model_kn.predict(X_noise_test)

print(metrics.classification_report(expected_kn_2, predicted_kn_2))

                  precision    recall  f1-score   support

          Одесса       1.00      1.00      1.00         1
               а       0.85      0.85      0.85        13
         автобус       0.00      0.00      0.00         1
       автозавод       1.00      1.00      1.00         1
       автолавка       1.00      1.00      1.00         1
         агрегат       0.33      1.00      0.50         1
          актёра       1.00      1.00      1.00         1
         актёров       1.00      1.00      1.00         1
         артисты       1.00      1.00      1.00         1
            базу       1.00      1.00      1.00         1
             без       1.00      1.00      1.00         2
          берегу       1.00      1.00      1.00         1
             бит       0.67      1.00      0.80         2
            блюд       0.00      0.00      0.00         1
        больницу       1.00      1.00      1.00         1
         больных       1.00      1.00      1.00         1
            б

Y:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


"Дерево принятия решений"

In [16]:
from sklearn.tree import DecisionTreeClassifier

t_start = time()
model_dtk = DecisionTreeClassifier(max_depth=None)
model_dtk.fit(X_train, Y_train)
print(model_dtk)
t_finish = time()
print(t_finish - t_start)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
38.97705078125


In [ ]:
expected_dtk_2 = Y_train
predicted_dtk_2 = model_dtk.predict(X_train)

print(metrics.classification_report(expected_dtk_2, predicted_dtk_2))

In [ ]:
expected_dtk = Y_test
predicted_dtk = model_dtk.predict(X_test)

print(metrics.classification_report(expected_dtk, predicted_dtk))

In [ ]:
expected_dtk_1 = Y_noise_test
predicted_dtk_1 = model_dtk.predict(X_noise_test)

print(metrics.classification_report(expected_dtk_1, predicted_dtk_1))

Метод "случайного леса"

In [39]:
from sklearn.ensemble import RandomForestClassifier

t_start = time()
model_rf = RandomForestClassifier(n_estimators=1, max_depth=None)
model_rf.fit(X_train, Y_train)
print(model_rf)
t_finish = time()
print(t_finish - t_start)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
1.5026791095733643


In [40]:
expected_rf = Y_test
predicted_rf = model_rf.predict(X_test)

print(metrics.classification_report(expected_rf, predicted_rf))

                  precision    recall  f1-score   support

          Одесса       0.50      1.00      0.67         1
               а       0.75      0.69      0.72        13
         автобус       1.00      1.00      1.00         1
       автозавод       1.00      1.00      1.00         1
       автолавка       1.00      1.00      1.00         1
         агрегат       1.00      1.00      1.00         1
          актёра       0.50      1.00      0.67         1
         актёров       1.00      1.00      1.00         1
         артисты       1.00      1.00      1.00         1
            базу       0.00      0.00      0.00         1
             без       0.00      0.00      0.00         2
          берегу       1.00      1.00      1.00         1
             бит       1.00      1.00      1.00         2
            блюд       0.50      1.00      0.67         1
        больницу       1.00      1.00      1.00         1
         больных       1.00      1.00      1.00         1
            б

Y:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [41]:
expected_rf_1 = Y_noise_test
predicted_rf_1 = model_rf.predict(X_noise_test)

print(metrics.classification_report(expected_rf_1, predicted_rf_1))

                  precision    recall  f1-score   support

          Одесса       0.50      1.00      0.67         1
               а       0.75      0.69      0.72        13
         автобус       1.00      1.00      1.00         1
       автозавод       1.00      1.00      1.00         1
       автолавка       1.00      1.00      1.00         1
         агрегат       1.00      1.00      1.00         1
          актёра       0.00      0.00      0.00         1
         актёров       0.00      0.00      0.00         1
         артисты       1.00      1.00      1.00         1
            базу       0.00      0.00      0.00         1
             без       0.00      0.00      0.00         2
          берегу       1.00      1.00      1.00         1
             бит       0.67      1.00      0.80         2
            блюд       0.00      0.00      0.00         1
        больницу       1.00      1.00      1.00         1
         больных       0.00      0.00      0.00         1
            б

Y:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [42]:
expected_rf_2 = Y_train
predicted_rf_2 = model_rf.predict(X_train)

print(metrics.classification_report(expected_rf_2, predicted_rf_2))

                  precision    recall  f1-score   support

          Одесса       0.73      0.80      0.76        10
               а       0.81      0.90      0.85       130
         автобус       1.00      0.90      0.95        10
       автозавод       1.00      0.90      0.95        10
       автолавка       1.00      1.00      1.00        10
         агрегат       1.00      1.00      1.00        10
          актёра       0.82      0.90      0.86        10
         актёров       1.00      1.00      1.00        10
         артисты       1.00      1.00      1.00        10
            базу       0.83      0.50      0.62        10
             без       0.84      0.80      0.82        20
          берегу       1.00      1.00      1.00        10
             бит       1.00      0.85      0.92        20
            блюд       0.70      0.70      0.70        10
        больницу       0.91      1.00      0.95        10
         больных       1.00      1.00      1.00        10
            б

Метод "опорных векторов"

In [25]:
from sklearn.svm import SVC

model_svc = SVC()
model_svc.fit(X_train, Y_train)
print(model_svc)

expected_svc = Y_test
predicted_svc = model.predict(X_test)

print(metrics.classification_report(expected_svc, predicted_svc))
print(metrics.confusion_matrix(expected_svc, predicted_svc))

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
                  precision    recall  f1-score   support

          Одесса       1.00      1.00      1.00         1
               а       0.56      0.69      0.62        13
         автобус       0.50      1.00      0.67         1
       автозавод       1.00      1.00      1.00         1
       автолавка       1.00      1.00      1.00         1
         агрегат       0.00      0.00      0.00         1
          актёра       0.00      0.00      0.00         1
         актёров       0.00      0.00      0.00         1
         артисты       1.00      1.00      1.00         1
            базу       0.00      0.00      0.00         1
             без       0.50      0.50      0.50         2
          берегу       0.00      0.00      0.00         1
             бит    

Y:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Нейронные сети

Обработка и нормализация

In [44]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.metrics import roc_auc_score
from keras.backend import clear_session

X_train_n = np.array(X_train)
Y_train_n = np.array(Y_array)
X_test_n = np.array(X_test)
Y_test_n = np.array(Y_array_test)
X_noise_test_n = np.array(X_noise_test)

from sklearn.preprocessing import MinMaxScaler 

min_max_scaler = MinMaxScaler() 

X_train_n = min_max_scaler.fit_transform(X_train_n) 
X_test_n = min_max_scaler.transform(X_test_n)
X_noise_test_n = min_max_scaler.transform(X_noise_test)

In [ ]:
Y_unic = {}
i = 0

for Y in Y_train:
    if Y in Y_unic:
        pass
    else:
        Y_unic[Y] = i
        i += 1

In [ ]:
Y_array = np.zeros([len(Y_train), len(Y_unic)])

for i in range(len(Y_train)):
    Y_array[i, Y_unic[Y_train[i]]] = 1

In [ ]:
Y_array_test = np.zeros([len(Y_test), len(Y_unic)])

for i in range(len(Y_test)):
    Y_array_test[i, Y_unic[Y_test[i]]] = 1

In [ ]:
Y_array_test_n = np.zeros([len(Y_noise_test), len(Y_unic)])

for i in range(len(Y_noise_test)):
    Y_array_test_n[i, Y_unic[Y_test[i]]] = 1

Обучение

In [128]:
model_neuron_1 = Sequential()

model_neuron_1.add(Dense(5, activation='softmax'))
model_neuron_1.add(Dense(5, activation='softmax'))
model_neuron_1.add(Dense(5, activation='softmax'))
model_neuron_1.add(Dense(462, activation='softmax'))

model_neuron_1.compile(loss='binary_crossentropy', optimizer='Adamax')

model_neuron_1.fit(np.array(X_train), Y_train_n, epochs=4,  verbose=2)

predictions_1_2 = model_neuron_1.predict(np.array(X_train_n))
print('Accuracy train: {}'.format(roc_auc_score(y_true=Y_train_n, y_score=predictions_1_2)))
predictions = model_neuron_1.predict(np.array(X_test_n))
print('Accuracy test: {}'.format(roc_auc_score(y_true=Y_test_n, y_score=predictions)))
predictions_1_1 = model_neuron_1.predict(np.array(X_noise_test_n))
print('Accuracy noise_test: {}'.format(roc_auc_score(y_true=Y_test_n, y_score=predictions_1_1)))

Epoch 1/4
 - 7s - loss: 0.0154
Epoch 2/4
 - 2s - loss: 0.0153
Epoch 3/4
 - 2s - loss: 0.0152
Epoch 4/4
 - 2s - loss: 0.0151
Accuracy train: 0.5121939169512718
Accuracy test: 0.5208329636372195
Accuracy noise_test: 0.5094552836402464
